In [1]:
import itertools
import re
import math

from functools import lru_cache
from collections import Counter, defaultdict

import numpy as np
import networkx as nx

from aocd import submit, get_data

DIRECTIONS = ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1))

def str2intList(s, sep="\n"):  # str2intList("2\n3") -> [2,3]
    return [int(x) if x.isdigit() else None
            for x in s.strip().split(sep)]

def imag2tup(i: complex) -> tuple: # 1+2j -> (1, 2)rd][Stats]
    return (int(i.real), int(i.imag))

def tup2imag(t: tuple) -> complex:
    return t[0] + t[1] * 1j

def level(method):
    def annotation(day, quiet=True, print_res=True, do_example=True):
        def solver(solve):
            part = 'ab'[method]
            if(testdata[day] and do_example):
                if type(testdata[day]) == list:
                    esol = solve(testdata[day][method], method=method)
                else:
                    esol = solve(testdata[day], method=method)
                if print_res:
                    print(f"Ex. {day}{part}", esol, sep="\t")
            sol = solve(get_data(day=day), method=method)
            if print_res:
                print(f"Lvl {day}{part}", sol, sep="\t")
            submit(sol, part, day=day, quiet=quiet)
            return solve # return the original function, so you can put another annotation on it
        return solver # 
    return annotation

level_a, level_b = level(0), level(1)
def level_ab(lvl):
    def solver(solve):
        level_a(lvl)(solve)
        level_b(lvl)(solve)
    return solver

testdata = ['',
'1721\n979\n366\n299\n675\n1456',
'1-3 a: abcde\n1-3 b: cdefg\n2-9 c: ccccccccc',
'..##.......\n#...#...#..\n.#....#..#.\n..#.#...#.#\n.#...##..#.\n..#.##.....\n.#.#.#....#\n.#........#\n#.##...#...\n#...##....#\n.#..#...#.#',
'ecl:gry pid:860033327 eyr:2020 hcl:#fffffd\nbyr:1937 iyr:2017 cid:147 hgt:183cm\n\niyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884\nhcl:#cfa07d byr:1929\n\nhcl:#ae17e1 iyr:2013\neyr:2024\necl:brn pid:760753108 byr:1931\nhgt:179cm\n\nhcl:#cfa07d eyr:2025 pid:166559648\niyr:2011 ecl:brn hgt:59in\neyr:1972 cid:100\nhcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926\n\niyr:2019\nhcl:#602927 eyr:1967 hgt:170cm\necl:grn pid:012533040 byr:1946\n\nhcl:dab227 iyr:2012\necl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277\n\nhgt:59cm ecl:zzz\neyr:2038 hcl:74454a iyr:2023\npid:3556412378 byr:2007\npid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980\nhcl:#623a2f\n\neyr:2029 ecl:blu cid:129 byr:1989\niyr:2014 pid:896056539 hcl:#a97842 hgt:165cm\n\nhcl:#888785\nhgt:164cm byr:2001 iyr:2015 cid:88\npid:545766238 ecl:hzl\neyr:2022\n\niyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719',
'FBFBBFFRLR',
'abc\n\na\nb\nc\n\nab\nac\n\na\na\na\na\n\nb',
'light red bags contain 1 bright white bag, 2 muted yellow bags.\ndark orange bags contain 3 bright white bags, 4 muted yellow bags.\nbright white bags contain 1 shiny gold bag.\nmuted yellow bags contain 2 shiny gold bags, 9 faded blue bags.\nshiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.\ndark olive bags contain 3 faded blue bags, 4 dotted black bags.\nvibrant plum bags contain 5 faded blue bags, 6 dotted black bags.\nfaded blue bags contain no other bags.\ndotted black bags contain no other bags.',
'nop +0\nacc +1\njmp +4\nacc +3\njmp -3\nacc -99\nacc +1\njmp -4\nacc +6',
'35\n20\n15\n25\n47\n40\n62\n55\n65\n95\n102\n117\n150\n182\n127\n219\n299\n277\n309\n576',
'16\n10\n15\n5\n1\n11\n7\n19\n6\n12\n4', 
'L.LL.LL.LL\nLLLLLLL.LL\nL.L.L..L..\nLLLL.LL.LL\nL.LL.LL.LL\nL.LLLLL.LL\n..L.L.....\nLLLLLLLLLL\nL.LLLLLL.L\nL.LLLLL.LL',
'F10\nN3\nF7\nR90\nF11', 
'939\n7,13,x,x,59,x,31,19',
['mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X\nmem[8] = 11\nmem[7] = 101\nmem[8] = 0',
'mask = 000000000000000000000000000000X1001X\nmem[42] = 100\nmask = 00000000000000000000000000000000X0XX\nmem[26] = 1'], 
'0,3,6',
['class: 1-3 or 5-7\nrow: 6-11 or 33-44\nseat: 13-40 or 45-50\n\nyour ticket:\n7,1,14\n\nnearby tickets:\n7,3,47\n40,4,50\n55,2,20\n38,6,12',
'class: 0-1 or 4-19\nrow: 0-5 or 8-19\nseat: 0-13 or 16-19\n\nyour ticket:\n11,12,13\n\nnearby tickets:\n3,9,18\n15,1,5\n5,14,9'],
'.#.\n..#\n###', 
'1 + 2 * 3 + 4 * 5 + 6', 
['0: 4 1 5\n1: 2 3 | 3 2\n2: 4 4 | 5 5\n3: 4 5 | 5 4\n4: "a"\n5: "b"\n\nababbb\nbababa\nabbbab\naaabbb\naaaabbb',
 '42: 9 14 | 10 1\n9: 14 27 | 1 26\n10: 23 14 | 28 1\n1: "a"\n11: 42 31\n5: 1 14 | 15 1\n19: 14 1 | 14 14\n12: 24 14 | 19 1\n16: 15 1 | 14 14\n31: 14 17 | 1 13\n6: 14 14 | 1 14\n2: 1 24 | 14 4\n0: 8 11\n13: 14 3 | 1 12\n15: 1 | 14\n17: 14 2 | 1 7\n23: 25 1 | 22 14\n28: 16 1\n4: 1 1\n20: 14 14 | 1 15\n3: 5 14 | 16 1\n27: 1 6 | 14 18\n14: "b"\n21: 14 1 | 1 14\n25: 1 1 | 1 14\n22: 14 14\n8: 42\n26: 14 22 | 1 20\n18: 15 15\n7: 14 5 | 1 21\n24: 14 1\n\nabbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa\nbbabbbbaabaabba\nbabbbbaabbbbbabbbbbbaabaaabaaa\naaabbbbbbaaaabaababaabababbabaaabbababababaaa\nbbbbbbbaaaabbbbaaabbabaaa\nbbbababbbbaaaaaaaabbababaaababaabab\nababaaaaaabaaab\nababaaaaabbbaba\nbaabbaaaabbaaaababbaababb\nabbbbabbbbaaaababbbbbbaaaababb\naaaaabbaabaaaaababaa\naaaabbaaaabbaaa\naaaabbaabbaaaaaaabbbabbbaaabbaabaaa\nbabaaabbbaaabaababbaabababaaab\naabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba'],
'Tile 2311:\n..##.#..#.\n##..#.....\n#...##..#.\n####.#...#\n##.##.###.\n##...#.###\n.#.#.#..##\n..#....#..\n###...#.#.\n..###..###\n\nTile 1951:\n#.##...##.\n#.####...#\n.....#..##\n#...######\n.##.#....#\n.###.#####\n###.##.##.\n.###....#.\n..#.#..#.#\n#...##.#..\n\nTile 1171:\n####...##.\n#..##.#..#\n##.#..#.#.\n.###.####.\n..###.####\n.##....##.\n.#...####.\n#.##.####.\n####..#...\n.....##...\n\nTile 1427:\n###.##.#..\n.#..#.##..\n.#.##.#..#\n#.#.#.##.#\n....#...##\n...##..##.\n...#.#####\n.#.####.#.\n..#..###.#\n..##.#..#.\n\nTile 1489:\n##.#.#....\n..##...#..\n.##..##...\n..#...#...\n#####...#.\n#..#.#.#.#\n...#.#.#..\n##.#...##.\n..##.##.##\n###.##.#..\n\nTile 2473:\n#....####.\n#..#.##...\n#.##..#...\n######.#.#\n.#...#.#.#\n.#########\n.###.#..#.\n########.#\n##...##.#.\n..###.#.#.\n\nTile 2971:\n..#.#....#\n#...###...\n#.#.###...\n##.##..#..\n.#####..##\n.#..####.#\n#..#.#..#.\n..####.###\n..#.#.###.\n...#.#.#.#\n\nTile 2729:\n...#.#.#.#\n####.#....\n..#.#.....\n....#..#.#\n.##..##.#.\n.#.####...\n####.#.#..\n##.####...\n##..#.##..\n#.##...##.\n\nTile 3079:\n#.#.#####.\n.#..######\n..#.......\n######....\n####.#..#.\n.#...#.##.\n#.#####.##\n..#.###...\n..#.......\n..#.###...', 
'mxmxvkd kfcds sqjhc nhms (contains dairy, fish)\ntrh fvjkl sbzzf mxmxvkd (contains dairy)\nsqjhc fvjkl (contains soy)\nsqjhc mxmxvkd sbzzf (contains fish)',
'', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ];

In [246]:
@level_ab(1)
def solve(data, method=0):
    data = sorted(str2intList(data))
    for line in itertools.product(data, repeat=method+2):
        if np.sum(line) == 2020:
            return np.prod(line)

Ex. 1a	514579
Lvl 1a	996996
Ex. 1b	241861950
Lvl 1b	9210402


In [3]:
@level_ab(2)
def solve(data, method):
    cnt = 0
    for line in data.split("\n"):
        nr, letter, pw = line.split(" ")
        letter = letter[0]
        start, end = map(int, nr.split("-"))
        if method == 0: cnt += pw.count(letter[0]) in range(int(start), int(end)+1)
        else:           cnt += (pw[start-1]+pw[end-1]).count(letter) == 1
    return cnt

Ex. 2a	2
Lvl 2a	519
Ex. 2b	1
Lvl 2b	708


In [4]:
@level_ab(3)
def solve(data, method=0):
    def sol(data, mov):
        d = np.array(list(map(list, data.split("\n")))) == '#'
        pos = cnt = x = 0
        while True:
            pos += mov
            x, y = imag2tup(pos)
            if x >= d.shape[0]:
                return cnt
            cnt += d[x, y % d.shape[1]]
    
    if method: return np.prod([sol(data, mov=mov) for mov in [1+1j, 1+3j, 1+5j, 1+7j, 2+1j]])
    else:      return sol(data, mov=1+3j)

Ex. 3a	7
Lvl 3a	214
Ex. 3b	336
Lvl 3b	8336352024


In [5]:
@level_ab(4)
def solve(data, method=0):
    ctr = 0
    ALLOWED_DICT = {
    "byr": lambda a: int(a) in range(1920, 2003), # (Birth Year) - four digits; at least 1920 and at most 2002.
    "iyr": lambda a: int(a) in range(2010, 2021), # (Issue Year) - four digits; at least 2010 and at most 2020.
    "eyr": lambda a: int(a) in range(2020, 2031), # (Expiration Year) - four digits; at least 2020 and at most 2030.
    "hgt": lambda a: (len(a) > 2) and (a[-2:] == "cm" and int(a[:-2]) in range(150, 194)) 
        or (a[-2:] == "in" and int(a[:-2]) in range(59, 77)), #(Height) - a number followed by either cm or in:        If cm, the number must be at least 150 and at most 193.        If in, the number must be at least 59 and at most 76.
    "hcl": lambda a: len(a) == 7 and a[0] == "#" and set(a[1:]).issubset(set("0123456789abcdef")), # (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    "ecl": lambda a: a in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"], # (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    "pid": lambda a: len(a) == 9 and set(a).issubset(set("0123456789")), # (Passport ID) - a nine-digit number, including leading zeroes.
    }
    for line in data.split("\n\n"):
        ut = dict([token.split(":") for token in re.split("[ \n]+", line)])
        if method: # part b
            stats = [k in ut and v(ut[k]) for k, v in ALLOWED_DICT.items()]
            ctr += all(stats)
        else: # part a
            ctr += set(ut).issubset(set(ALLOWED_DICT))
    return ctr

Ex. 4a	4
Lvl 4a	125
Ex. 4b	6
Lvl 4b	116


In [6]:
@level_ab(5)
def solve(data, method):
    data = data.replace("F", "0").replace("B", "1").replace("L", "0").replace("R", "1")
    data = [int(d, 2) for d in data.split("\n")]
    if len(data) <= 1: return
    elif method:       return (set(range(min(data), max(data))) - set(data)).pop()
    else:              return max(data)

Ex. 5a	None
Lvl 5a	801
Ex. 5b	None
Lvl 5b	597


In [7]:
@level_ab(6)
def solve(data, method=0):
    cnt = 0
    for section in data.split("\n\n"):
        nr_lines = section.count("\n") + 1
        if method: cnt += sum([lcnt == nr_lines and l != "\n" for l, lcnt in Counter(section).items()])
        else:      cnt += len(set(section).difference({"\n"}))
    return cnt

Ex. 6a	11
Lvl 6a	6416
Ex. 6b	6
Lvl 6b	3050


In [8]:
@level_ab(7)
def solve(data, method=0):
    def dfs(G, start, factor):
        return sum([dfs(G, follower, factor * val["cnt"]) + factor * val["cnt"] 
                    for follower, val in G[start].items()])

    G = nx.DiGraph()
    for line in data.split("\n"):
        line = re.sub("(bags?)", "", line)
        containing, contained = line[:-2].split("  contain ")
        if not "no other" in contained:
            for contain in contained.split(" , "):
                cspl = contain.split(" ")
                cnt = int(cspl[0]) if len(cspl) == 3 else 1
                contain = " ".join(cspl[-2:])
                G.add_edge(containing, contain, cnt=cnt)
    if method: return dfs(G, "shiny gold", 1)
    else:      return len(nx.algorithms.dag.ancestors(G, "shiny gold"))        

Ex. 7a	4
Lvl 7a	252
Ex. 7b	32
Lvl 7b	35487


In [9]:
@level_ab(8)
def solve(data, method=0):
    def interpret(lines):
        seen = set()
        acc = ptr = 0
        for icnt in range(10000):
            # print(ptr, acc, lines[ptr])
            if (ptr in seen and method==0) or ptr >= len(lines): return acc
            
            instr, arg = lines[ptr]
            if instr=="acc":   acc+=int(arg)
            elif instr=="jmp": ptr+=int(arg)-1
                
            seen.add(ptr)
            ptr+=1
        return False
            
    lines = [line.split(" ") for line in data.split("\n")]
    if method:
        for i in range(len(lines)):
            lc = [l[:] for l in lines]
            swap = {"nop": "jmp", "jmp": "nop", "acc": "acc"}
            lc[i][0] = swap[lc[i][0]]
            if lc[i][0] != "acc":
                res = interpret(lc)
                if res is not False:
                    return res
    return interpret(lines)

Ex. 8a	5
Lvl 8a	1614
Ex. 8b	8
Lvl 8b	1260


In [10]:
@level_ab(9)
def solve(data, method=0):
    d = str2intList(data)
    preamble_len = 5 if len(d) == 20 else 25 # if test-set, we have shorter preamble
    for i in range(preamble_len, len(d)):
        if d[i] not in {sum(tup) for tup in itertools.permutations(d[i-preamble_len:i], 2)}:
            if method:
                number_to_find = d[i]
                break
            return d[i] # method 0 returns here
    for i in range(len(d)):
        for lookback in range(i+1):
            sub_array = d[i-lookback:i]
            if sum(sub_array) == number_to_find:
                return min(sub_array) + max(sub_array)

Ex. 9a	127
Lvl 9a	26134589
Ex. 9b	62
Lvl 9b	3535124


In [11]:
@lru_cache
def dfs(g, u, t):
    return u==t or sum(dfs(g, c, t) for c in range(u+1, u+4) if c in g)

@level_ab(10)
def solve(data, method=0):
    data = [0]+sorted(str2intList(data))
    data += [max(data)+3]
    if method:
        return dfs(tuple(data), 0, max(data))
    else:
        cd = Counter(np.diff(data))
        return cd[1]*cd[3]

Ex. 10a	35
Lvl 10a	2030
Ex. 10b	8
Lvl 10b	42313823813632


In [12]:
from numba import njit
FLOOR, EMPTY, OCCUPIED = [0, 1, 2]

@njit()
def neighbors(dc, i, j, dist):
    im, jm = dc.shape
    cnt = 0
    for ip, jp in DIRECTIONS:
        for d in range(1, dist):
            ipos, jpos = i+ ip * d, j + jp * d
            if 0 <= ipos < im and 0 <= jpos < jm:
                if (dcp := dc[ipos][jpos]) == EMPTY: break
                elif dcp == OCCUPIED: cnt += 1; break
            else: break
    return cnt

@njit()
def solve10(data, method):
    im, jm = data.shape
    dc = np.zeros_like(data, dtype=data.dtype)
    while not (dc == data).all():
        dc, data = data, dc  # swap without extra allocations
        for i in range(im):
            for j in range(jm):
                if (curr := dc[i][j]) != FLOOR:
                    neigh = neighbors(dc, i, j, 11 if method else 2)
                    if curr == OCCUPIED and neigh >= 4 + method: curr = EMPTY
                    elif curr == EMPTY and not neigh:            curr = OCCUPIED
                    data[i][j] = curr
    return (dc == OCCUPIED).sum()

@level_ab(11)
def solve(data, method=0): # numba can't optimize parsing, so we'll do it non-jitted
    data = np.array([[".L#".index(line[i]) for i in range(len(line))] for line in data.split("\n")])
    return solve10(data, method)

Ex. 11a	37
Lvl 11a	2483
Ex. 11b	26
Lvl 11b	2283


In [13]:
dire = {"N": 1, "S": -1, "E": 1j, "W": -1j}
turn = {"L": -1j, "R": 1j}

@level_ab(12)
def solve(data, method=0):
    pos, wp = 0, 1+10j if method else 1j
    for line in data.split("\n"):
        cmd, dist = line[0], int(line[1:])
        if cmd in dire:
            if method:    wp  += dist * dire[cmd]
            else:         pos += dist * dire[cmd]
        elif cmd == "F":  pos += dist * wp
        elif cmd in turn: wp *= turn[cmd] ** (dist//90)
        # print("cmd", line, "pos", pos, "facing", facing)
    return int(abs(pos.real)+abs(pos.imag))

Ex. 12a	25
Lvl 12a	845
Ex. 12b	286
Lvl 12b	27016


In [14]:
from sympy.ntheory.modular import crt

@level_ab(13)
def solve(data, method=0):
    offset, times = data.split("\n")
    offset, times = int(offset), str2intList(times, sep=",")
    if method: # x % offset_i == -i
        mod_rem = {t: -i for i, t in enumerate(times) if t}
        return int(crt(mod_rem.keys(), mod_rem.values())[0]) # sympy.crt returns (mpz, unrelated-int)
    else: # some t can be None. -offset % t yields a positive number
        return np.prod(min([[-offset % t, t] for t in times if t]))

Ex. 13a	295
Lvl 13a	4782
Ex. 13b	1068781
Lvl 13b	1118684865113056


In [15]:
@level_ab(14)
def solve(data, method=0):
    memory = {}
    mask = "X0"[method]
    for line in data.split("\n"):
        if line.startswith("mask = "):
            bitmask = line[7:]
        elif line.startswith("mem"):
            addr, val = re.findall("[0-9]+", line)
            to_use = addr if method else val
            padded = bin(int(to_use))[2:].rjust(36, "0")
            masked = [(a if b==mask else b) for a, b in zip(padded, bitmask)]
            if method: # mask to list of lists with len(inner_list) == 0 or 1
                for p in itertools.product(*[["0", "1"] if letter == "X" else [letter] for letter in masked]):
                    memory[int("".join(p), 2)] = int(val)  # write multiple memory positions
            else:
                memory[int(addr)] = int("".join(masked), 2)
    return sum(memory.values())

Ex. 14a	165
Lvl 14a	6631883285184
Ex. 14b	208
Lvl 14b	3161838538691


In [16]:
from numba import njit

@njit()
def solve_(spoken, method):
    seen = dict() # can't init as a 1-liner due to numba
    for i, nr in enumerate(spoken):
        seen[nr] = i+1
    number = 0
    for i in range(len(spoken)+1, 30_000_000 if method else 2020):
        # same time assignment: use old number to set, then set number, using old number
        seen[number], number = i, (i - seen[number]) if number in seen else 0
    return number

@level_ab(15)
def solve(data, method=0):
    # numba doesn't support int(), so parsing must be done outside optimized code
    return solve_(tuple(str2intList(data, sep=",")), method)

Ex. 15a	436
Lvl 15a	610
Ex. 15b	175594
Lvl 15b	1407


In [17]:
@level_ab(16)
def solve(data, method=0):
    allow, your, other = data.split("\n\n") # parsing
    your = str2intList(your.split("\n")[1], sep=",")
    other = np.array([str2intList(o, sep=",") for o in other.split("\n")[1:]])
    
    allowed = defaultdict(set) # build a name: allowed set
    for line in allow.split("\n"):
        name, options = line.split(": ")
        for option in options.split(" or "):
            start, end = str2intList(option, sep="-")
            allowed[name].update(range(int(start), int(end)+1))
    all_allowed = set.union(*allowed.values())
    
    if method: # throw away invalid lines first
        other = np.array([line for line in other if all_allowed.issuperset(set(line))]) 
        g = nx.Graph() # build name: colid graph, do bipartite matching for assignment problem
        for colid, col in enumerate(other.T):
            for name, nallowed in allowed.items():
                if nallowed.issuperset(col):
                    g.add_edge(name, colid)
        matches = nx.algorithms.bipartite.matching.minimum_weight_full_matching(g).items()
        return np.prod([your[colid] for name, colid in matches
                            if type(name) == str and name.startswith("departure")], dtype=int)
    return sum(filter(lambda o: o not in all_allowed, other.flat)) # else not needed

Ex. 16a	71
Lvl 16a	23036
Ex. 16b	1
Lvl 16b	1909224687553


In [18]:
from scipy.signal import convolve

@level_ab(17)
def solve(data, method=0, it=6): # run with 6 iterations by default
    field = np.array([[list(line) for line in data.split("\n")]]) == "#"
    if method: field = np.expand_dims(field, 0)
    is_alive = np.vectorize(lambda x: x in [3, 1002, 1003])    
    convfilt = np.ones((3,)*(method+3), dtype=int)
    convfilt[(1,)*(method+3)] = 1000 # set center 0
    for _ in range(it):
        field = is_alive(convolve(field.astype(int), convfilt, "full"))
    return field.sum()

Ex. 17a	112
Lvl 17a	310
Ex. 17b	848
Lvl 17b	2056


In [247]:
from more_itertools import pairwise
P_PAR = r"\([ \d+*]+\)" # parenthesis regex
P_ADD = r"\d+ \+ \d+"   # addition regex

def subproblem(line, method):
    while     (sub := re.search(P_PAR, line)):
        line =     line.replace(sub.group(), subproblem(sub.group()[1:-1], method))
    if method: # do add after brackets if method
        while (sub := re.search(P_ADD, line)) and sub.group() != line:
            line = line.replace(sub.group(), subproblem(sub.group()      , method))
    spl = line.split(" ")
    acc = int(spl[0])
    for op, num in pairwise(spl[1:]): # offset from 0th element, in 2-steps
        if   op == "+": acc += int(num)
        elif op == "*": acc *= int(num)
    return str(acc)

@level_ab(18)
def solve(data, method=0): # run with 6 iterations by default
    return sum([int(subproblem(line, method)) for line in data.split("\n")])

Ex. 18a	71
Lvl 18a	3348222486398
Ex. 18b	231
Lvl 18b	43423343619505


In [3]:
from lark import Lark

@level_ab(19)
def solve(data, method=0):
    rules, examples = data.split("\n\n")
    if method:
        rules = re.sub(r"\n8:.+", "\n8: 42 | 42 8", rules)
        rules = re.sub(r"\n11:.+", "\n11: 42 31 | 42 11 31", rules)
    rules = re.sub(r"\d+", lambda d: "start" if d.group()=="0" else "w"+d.group(), rules)
    parser = Lark('%ignore " "\n' + rules)
    
    cnt = 0
    for example in examples.split("\n"):
        try: parser.parse(example); cnt+=1
        except: pass        
    return cnt

Ex. 19a	2
Lvl 19a	208
Ex. 19b	12
Lvl 19b	316


In [259]:
def variations(img):
    for rot in range(4):  # add connections for all
        for _ in range(2):
            yield img
            img = np.fliplr(img)
        img = np.rot90(img)


class Image:
    def __init__(self, img_as_txt: str):
        name, *img = img_as_txt.split("\n")
        self.id = int(name.replace("Tile ", "")[:-1])
        self.img = (np.array([list(i) for i in img]) == '#').astype(int)
        self.corners = {tuple(var[0]) for var in variations(self.img)}
        self.neigh = set()
        self.pos_neigh = dict()
        self.visited = False

    def edge(self, i: complex) -> np.array:
        if   i == -1:  return self.img[0]
        elif i == 1:   return self.img[-1]
        elif i == -1j: return self.img[:, 0]
        elif i == 1j:  return self.img[:, -1]

    def visit(self, pos = None, must_match = None):
        if self.pos_neigh: return  # will bet set in visiting
        elif pos:   # rotate until we match
            for v in variations(self.img):
                self.img = v
                if (self.edge(pos) == must_match).all():
                    break 
        for pos in [1j**i for i in range(4)]:
            for neigh in self.neigh:
                if tuple(self.edge(pos)) in neigh.corners:
                    self.pos_neigh[-pos] = neigh
                    neigh.visit(-pos, must_match=self.edge(pos))

    def walk(self, direction: complex):
        yield self
        if direction in self.pos_neigh:
            yield from self.pos_neigh[direction].walk(direction)

@level_ab(20)
def solve(data, method=0):
    images = [Image(img_as_txt) for img_as_txt in data.split("\n\n")]
    for curr, other in itertools.permutations(images, 2):
        if set(curr.corners).intersection(other.corners):
            curr.neigh.add(other)
    if not method:
        images = sorted(images, key=lambda i: len(i.neigh))
        return np.prod([i.id for i in images[:4]])
    center = max(images, key=lambda img: len(img.neigh))
    center.visit()
    top_left = list(list(center.walk(1))[-1].walk(1j))[-1]
    field = np.vstack([np.hstack([row.img[1:-1, 1:-1] for row in col.walk(-1j)]) for col in top_left.walk(-1)])
    monster = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """
    mask = (np.array([list(line) for line in monster.split("\n")]) == "#").astype(int)

    for fieldv in variations(field):  # for all field-variations and mask shifts
        fcnt = 0
        fx, fy = field.shape
        mx, my = mask.shape
        for i, j in itertools.product(range(fx-mx), range(fx-my)):
            fcnt += (fieldv[i:i+mx, j:j+my] & mask == mask).all()


© Copyright 2004-2020, NetworkX Developers Last updated on Aug 22, 2020.
Built with Sphinx using a theme provided by Read the Docs.
        if fcnt: return fieldv.sum() - mask.sum() * fcnt

Ex. 20a	20899048083289
Lvl 20a	18449208814679
Ex. 20b	273
Lvl 20b	1559


In [497]:
@level_ab(21)
def solve(data, method=0):
    g = nx.Graph()
    all_ing_ctr = Counter()
    for line in data.split("\n"):
        ing, alle = line.split(" (contains ")
        ing = set(ing.split(" "))
        alle = set(alle[:-1].split(", "))
        all_ing_ctr.update(ing)
        for a, i in itertools.product(alle, ing):
            if g.has_edge(a,i):  # the solution isn't unique so we have to use this trick
                g.get_edge_data(a,i)["weight"] -= 1
            else: 
                g.add_edge(a, i, weight=0)
    
    all_ing = set(all_ing_ctr)
    match = nx.bipartite.minimum_weight_full_matching(g)
    unsafe = set(match)  # dict, mapping unsafe ingredients AND compounds
    if method: return ",".join((sorted(all_ing & unsafe, key=lambda ing: match[ing])))
    else:      return sum([all_ing_ctr[s] for s in all_ing - unsafe])

Ex. 21a	5
Lvl 21a	2542
Ex. 21b	mxmxvkd,sqjhc,fvjkl
Lvl 21b	hkflr,ctmcqjf,bfrq,srxphcm,snmxl,zvx,bd,mqvk
